In [7]:
import json

'''loads the url list
'''
# open up the url_list
with open('C:/Projects/Housing_Price_Prediction/data_processing/url_exts.json', 'r') as f:
    raw_url_ext_list = f.read()

url_ext_list = json.loads(raw_url_ext_list)

# **FIX PLS repeats in the urls probably from the problems in the rent interval ;.; going to fix later
url_ext_list = url_ext_list[0:540]

In [12]:
'''get's the building key from the url extensions and store it in a new param_list
'''
# get the building keys from the url
request_params_list = []
for url_ext in url_ext_list:
    # get the last url extension
    last_ext = url_ext.split('/')[-2]

    # if it's not already the building key, get the building key from the last url extension
    split_last_ext_list = last_ext.split('-')
    if len(split_last_ext_list) > 1:
        if "_" not in split_last_ext_list[-1]:
            building_key = split_last_ext_list[-1]
    else:
        building_key = split_last_ext_list[0]

    # append the building key and the url to the param_list
    request_params_list.append((url_ext, building_key))

In [16]:
import requests

'''makes html requests to each listing's page and extracts information from zillow's backend server.  Then stores the responses in a response list to parse later
'''

# make a request for each listing and store the response in the response_list
response_list = []
for (url_ext, building_key) in request_params_list:

    # the `header` parameter includes information about your browser's current status.  important for preventing captia prompts from zillow
    headers = {
        'authority': 'www.zillow.com',
        'accept': '*/*',
        'accept-language': 'en-US,en;q=0.9,it-IT;q=0.8,it;q=0.7',
        'client-id': 'vertical-living',
        'content-type': 'text/plain',
        'dnt': '1',
        'origin': 'https://www.zillow.com',
        'referer': 'https://www.zillow.com' + url_ext,
        'sec-ch-ua': '"Chromium";v="112", "Google Chrome";v="112", "Not:A-Brand";v="99"',
        'sec-ch-ua-mobile': '?1',
        'sec-ch-ua-platform': '"Android"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'user-agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Mobile Safari/537.36',
    }

    # the `data` parameter includes info on the building query to zillow's grapql database.  the necessary component to include is the `buiding_key` which was extracted from the url extensions
    data = '{"operationName":"BuildingQuery","variables":{"buildingKey":' + "\"" + building_key + "\"" + ',"cache":false,"latitude":null,"longitude":null,"lotId":null,"update":true},"extensions":{"persistedQuery":{"version":1,"sha256Hash":"a6b5cd248233c9b3da074a0c6daacf6e6388a9b3a7de3347043b2e1b41d095b6"}}}'

    response = requests.post('https://www.zillow.com/graphql/', headers=headers, data=data)
    response_list.append(response)

In [8]:
import json

'''iterate through each response in response_list and performs a few steps:
1. parse it into a python dict
2. Extract the `building` key from the dict 
3. appends it to the `listing_info_list` to process later.

The object from the reponse represents a dict of dicts, and the dictionary that contains the relevent listing information is in the `building` dictionary.  This block iterates through each response and extracts that `building` dictionary to parse later
'''
listing_info_list = []
for response in response_list:

    # parses the response into the raw listing info
    raw_listing_info_dict = json.loads(response.text)

    # iterate through the dict of dicts until the 'building' key is found
    for key, value in raw_listing_info_dict.items():
        if key == 'data':
            for key, value in value.items():
                if key == 'building':
                    listing_info_dict = value
                    
    # finally append the listing information to the listing information list
    listing_info_list.append(listing_info_dict)
#print(dict.keys())


dict_keys(['building', 'viewer', 'abTests'])
dict_keys(['regionIds', 'adTargets', '__typename', 'breadcrumbs', 'buildingName', 'streetAddress', 'latitude', 'longitude', 'streetViewTileImageUrlLocationModuleLatLong', 'streetViewTileImageUrlLocationModuleAddress', 'lotId', 'streetViewMetadataUrlMediaWallLatLong', 'streetViewMetadataUrlMediaWallAddress', 'mapTileGoogleMapUrlLocationModule', 'mapTileGoogleMapUrlFullWidthMax', 'address', 'streetViewTileImageUrlHalfWidthLatLong', 'streetViewTileImageUrlHalfWidthAddress', 'city', 'state', 'zipcode', 'isWaitlisted', 'contactInfo', 'isLandlordLiaisonProgram', 'buildingType', 'isInstantTourEnabled', 'isInstantTourCancellable', 'bestGuessTimezone', 'rentalInstantTour', 'listingFeatureType', 'photoCount', 'amenitiesVRModels', 'videos', 'galleryPhotos', 'galleryAmenityPhotos', 'fullAddress', 'bdpUrl', 'zpid', 'buildingPhoneNumber', 'county', 'ungroupedUnits', 'floorPlans', 'buildingAttributes', 'nearbyCities', 'nearbyNeighborhoods', 'country', 'nea

In [16]:
useless_keys = ['regionIds', 'adTargets', '__typename', 'breadcrumbs', 'streetViewTileImageUrlLocationModuleLatLong', 'streetViewTileImageUrlLocationModuleAddress', 'streetViewMetadataUrlMediaWallLatLong', 'streetViewMetadataUrlMediaWallAddress', 'mapTileGoogleMapUrlLocationModule', 'mapTileGoogleMapUrlFullWidthMax', 'streetViewTileImageUrlHalfWidthLatLong', 'streetViewTileImageUrlHalfWidthAddress', 'isWaitlisted', 'isInstantTourEnabled', 'isInstantTourCancellable', 'bestGuessTimezone', 'rentalInstantTour', 'amenitiesVRModels', 'galleryPhotos', 'galleryAmenityPhotos', 'fullAddress', 'bdpUrl', 'zpid', 'buildingPhoneNumber', 'county', 'ungroupedUnits', 'nearbyCities', 'nearbyNeighborhoods', 'country', 'nearbyZipcodes', 'nearbyBuildingLinks', 'comps', 'ppcLink', 'reviewsInfo', 'housingConnector', 'localProtections', 'buildingRentalPremiumPackagesInfo', 'homeInsights', 'bestMatchedUnit', 'photos', 'amenityPhotos', 'staticMap', 'staticMapSatellite', 'streetViewLatLong', 'streetViewAddress', 'thirdPartyVirtualTours', 'currency', 'specialOffers', 'listingMetadata', 'vaLoanStatus', 'engrain', 'homeTypes', 'providerInfoList', 'unitsVRModels', 'city', 'state', 'contactInfo']
for key in useless_keys:
    info_list.pop(key, None)
list = []
list.append(info_list)
print(list)

[{'buildingName': 'The New Carroll', 'streetAddress': '305 Bellevue Ave E', 'latitude': 47.621158, 'longitude': -122.326944, 'lotId': '1001480426', 'address': {'streetAddress': '305 Bellevue Ave E', 'city': 'Seattle', 'state': 'WA', 'zipcode': '98102', '__typename': 'Address', 'neighborhood': None}, 'zipcode': '98102', 'isLandlordLiaisonProgram': False, 'buildingType': 'FOR_RENT', 'listingFeatureType': 'featured-ppc', 'photoCount': 12, 'videos': [], 'floorPlans': [{'zpid': '2060720283', '__typename': 'FloorPlan', 'units': None, 'floorplanVRModel': None, 'minPrice': 2850, 'maxPrice': 2850, 'availableFrom': '0', 'baths': 1, 'beds': 2, 'floorPlanUnitPhotos': [], 'name': '104', 'photos': None, 'sqft': 1150, 'thirdPartyVirtualTour': None, 'vrModels': [], 'maloneId': None, 'description': 'Smoking allowed:No'}], 'buildingAttributes': {'hasSharedLaundry': True, 'airConditioning': 'Unknown', 'appliances': ['Refrigerator', 'Freezer', 'Oven', 'Range', 'Dishwasher', 'GarbageDisposal'], 'parkingTyp